# Introduction

## Exploratory data analysis

Exploratory data analysis is an activity where you ....... *explore your data*. It's often conducted towards the beginning of a data science or analysis workflow and is an interactive process to build up your familiarity with the data; identify its structure and patterns; spot noise, errors, and missing values; and begin to formulate research questions and hypotheses.  

Chapter 7 of R for Data Science <a href="https://r4ds.had.co.nz/exploratory-data-analysis.html" target="_blank">(Wickham and Grolemund, 2017)</a> provides an excellant overview of techniques for exploratory data analysis. They suggest that two questions should guide your initial exploration of datasets:

* What type of variation occurs within variables?
* What type of covariation occurs between variables?

A variable is a property or feature of interest that can be measured and a value is the state of the variable when it was measured. Columns in a `DataFrame` or `GeoDataFrame` or bands in raster often correspond to variables and cells in a table or pixels in a raster correspond to values for an observation. 

### Task

Here, you will build on the data visualisation skills from the previous lab to explore crop yield data collected by a harvester in Western Australia. You will use data visualisations and summaries to identify noise or errors in the dataset and remove or clean them. In this lab you will learn to:

* generate summary tables and descriptive statistics
* visualise data distributions
* visualise relationships between variables
* identify and remove missing or noisy values


## Setup

### Run the labs

You can run the labs locally on your machine or you can use cloud environments provided by Google Colab. **If you're working with Google Colab be aware that your sessions are temporary and you'll need to take care to save, backup, and download your work.**

<a href="https://colab.research.google.com/github/data-analysis-3300-3003/colab/blob/main/lab-3-self-guided.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Download data

If you need to download the data for this lab, run the following code snippet. 

In [ ]:
import os

if "week-3" not in os.listdir(os.getcwd()):
    os.system('wget "https://github.com/data-analysis-3300-3003/data/raw/main/data/week-3.zip"')
    os.system('unzip "week-3.zip"')

### Working in Colab

If you're working in Google Colab, you'll need to install the required packages that don't come with the colab environment.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install geopandas
    !pip install pyarrow
    !pip install mapclassify
    !pip install rasterio
    !pip install libpysal
    !pip install esda
    !pip install splot

### Import modules

In [ ]:
# Import modules
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import plotly.io as pio
import esda

from splot.esda import plot_moran
from libpysal.weights import KNN, lag_spatial

# setup renderer
if 'google.colab' in str(get_ipython()):
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "jupyterlab"

## Load data

In [ ]:
# Load the crop yield data
crop_yield_data_path = os.path.join(os.getcwd(), "week-3")

# Get a list of crop yield data
crop_yield_data_files = os.listdir(crop_yield_data_path)

# Combine the geojson files into one GeoDataFrame
dfs = []

for i in crop_yield_data_files:
    if i.endswith(".geojson"):
        print(f"Loading file {i} into a Geopandas GeoDataFrame")
        tmp_df = gpd.read_file(os.path.join(crop_yield_data_path, i))
        dfs.append(tmp_df)

gdf = pd.concat(dfs, axis=0)

## Data summaries

An initial data exploration task is to produce summary statistics for the variables in our datasets. Pandas `DataFrame`s and GeoPandas `GeoDataFrame`s have a `describe()` method which generates a `DataFrame` of summary statistics for each variable. 

In [ ]:
# Describe our DataFrame of crop yield data
gdf.describe()

`describe()` returns to us a count of the number of observations in a variable, the mean value for observations in a variable, and summary statistics describing the distribution and range of values (standard deviation, percentiles (median = 50th percentile), and min and max values).

However, there are two main groups in our dataset: canola observations and wheat observations denoted by the `Variety` column. It will be more informative to generate summary statistcs for each group separately. We can do this using the Pandas `groupby()` function which splits a `DataFrame` into subsets based upon a grouping variable, computes statistics for each subset, and then combines the results. Here, we need to `groupby()` `Variety` to generate summary statistics for each crop type. 

We'll also generate these summary statistics within a context manager (denoted by a `with` block). This context allows us to change the default display values for a `DataFrame` only for this context without affecting the global defaults that apply to the rest of the notebook. This is a useful trick in case you have a particular need to control how a `DataFrame` is displayed (e.g. printing all rows as specified by the `display.max_rows` option).

In [ ]:
with pd.option_context("display.max_rows", None, "display.float_format", lambda x: "%.3f" % x):
    display(gdf.groupby(["Variety"]).describe())

This still isn't a very helpful layout to view the summary statistics as not all of the statistics can be displayed. Let's transpose the summary statistics so rows become columns and vice versa using the `T` transpose operator.

In [ ]:
with pd.option_context("display.max_rows", None, "display.float_format", lambda x: "%.3f" % x):
    display(gdf.loc[:, ["Variety", "DryYield", "gndvi", "ndyi"]].groupby(["Variety"]).describe().T)

## Data distributions

The mean tells us the average value for a variable. However, it is susceptible to outliers and extreme values. Therefore, it is important to view the mean and the median (50th percentile) together as the median is not affected by extreme values. 

However, neither the mean or the median reveal the spread or distribution of values for a variable. The min and max values tell us what the range of values are for a variable. This can be useful for detecting potential measurement error and noise (e.g. is the max value for wheat yield sensible?).

The inter-quartile range (difference between the 75th and 25th percentile values) tells us how spread out the data is around the median and the standard deviation tells us how spread out the data is around the mean. Assuming a normal distribution, ~68% of the values are within one standard deviation of the mean. 

It is often useful to visualise the distribution of variables. A histogram is a common visualisation for distributions. The height of the bars of a histogram correspond to the count of values that fall within the bin. The width of the bar corresponds to the bin width. 

#### Recap quiz

<details>
    <summary><b>What is a limitation of using min and max values to represent the distribition of values for a variable?</b></summary>
The min and max values can be affected by extreme values and don't tell us anything about the shape or density of the distribution of data values.
</details>

<p></p>

<details>
    <summary><b>If the standard deviation is small relative to the mean, what does this tell us about the spread of the data?</b></summary>
There is not much spread in the data away from the mean.
</details>

<p></p>


In [ ]:
fig = px.histogram(
    data_frame=gdf, 
    x="DryYield", 
    facet_col="Variety", 
    hover_data=["DryYield", "Elevation", "WetMass"])
fig.show()

The `histogram()` function from Plotly Express has a `nbins` parameter that can be used to specify the number of bins.

#### Recap quiz

**Use the Plotly Express `nbins` parameter of the `histogram()` function to change the number bins, and consequently the bin width, and explore how this affects the visualisation of the distribution.**

In [ ]:
## add code here ##

<details>
    <summary><b>answer</b></summary>
    
```{python}
fig = px.histogram(
    data_frame=gdf, 
    x="DryYield", 
    facet_col="Variety", 
    nbins=5, ### CHANGE THIS VALUE
    hover_data=["DryYield", "Elevation", "WetMass"])
fig.show()
```
</details>

<details>
    <summary><b>What happens if you generate a histogram with too large a bin width?</b></summary>
You will smooth out variation in the data and will not accurately reflect the shape of the distribution of the data.
</details>

<p></p>

<details>
    <summary><b>What happens if you generate a histogram with too small a bin width?</b></summary>
You might not be able to see the dominant shape of the distribution of data values as the histogram could appear "spikey" with local variability in the distribution of values emphasised.
</details>

## Subsetting pandas `DataFrame`s

To subset data from a pandas `DataFrame` we use the square brackets `[]` to specify the data values we'd like to extract. The `[]` operator can be thought of as a get item operation. 

**Selection by labels**

*Selection by labels* refers to selecting values from a `DataFrame` by their label (i.e. column name).

* To select a column from a `DataFrame` we pass the column name into `[]` (e.g. `dry_yield = gdf["DryYield"]` where `dry_yield` is a `Series` object).
* To select many columns from a `DataFrame` we pass a list of column names into `[]` (e.g. (e.g. `df_yield = gdf[["DryYield", "Variety"]]` where `df_yield` is a `DataFrame` object).

**Selection by position**

*Selection by position* refers to selecting values from a `DataFrame` by their index position (i.e. row or column number starting at 0).

* To select the $n^{th}$ row pass in `[n-1:n]`. Remember that Python indexes from zero so the $n-1$ index position is the $n^{th}$ row. The slice operator `:` is exclusive so `[n-1:n]` will only select the row at `n-1` (e.g. to select the $2^{nd}$ row use `df_row_2 = gdf[1:2]`).
* To select a slice of rows use the slice operator (e.g. to select the first 10 rows use `df_10_rows = gdf[0:10]`).

**Selection by condition**

*Selection by condition* selects rows that are `True` based on a condition (e.g. selecting all rows with a `DryYield` greater than 1.5 - `df_yield_gt_1_5 = gdf[gdf["DryYield"] > 1.5]`).

**`loc[]` and `iloc[]`**

The more robust approach to subsetting data from `DataFrame`s is using the `loc` and `iloc` methods, which also support multi-index selection (i.e. selecting rows and columns). 

* `loc` is used for selecting by labels (e.g. `df_yield = gdf.loc[:, ["DryYield", "Variety"]]` - note the `[:, ["DryYield", "Variety"]]` syntax where `:` means select all rows).
* `iloc` is used for selecting by position (e.g. to select the first 10 rows use `df_10_rows = gdf.iloc[0:10, :]`).
* To select the first 10 rows and columns we'd use `df_10_rows_10_cols = gdf.iloc[0:10, 0:10]`).

#### Recap quiz

<details>
    <summary><b>How many rows will the object referenced by <code>df_temp</code> have after calling <code>df_temp=df.iloc[0:5, :]</code>?</b></summary>
5 rows at index positions 0 to 4 from the <code>DataFrame</code> <code>df</code>.
</details>

<p></p>

<details>
    <summary><b>How many columns from <code>df</code> will the object referenced by <code>df_temp</code> have after calling <code>df_temp=df.iloc[0:5, :]</code>?</b></summary>
All the columns from <code>df</code>.
</details>

<p></p>

**Use `loc` and the `DataFrame` referenced by `gdf` to select all rows where `DryYield` is greater than 2.**

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer</b></summary>

```{python}
df_gt_2 = gdf.loc[(gdf["DryYield"] > 2), :]
df_gt_2.head()
```
</details>

<p></p>

These are some useful resources on subsetting pandas `DataFrame`s:

* pandas Getting Started: <a href="https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-specific-rows-and-columns-from-a-dataframe" target="_blank">How do I select specific rows and columns from a DataFrame?</a>
* The first few sections of the pandas docs on <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#selection-by-position" target="_blank">Indexing and selecting data</a>
* McKinney (2022) Python for Data Analysis - section on <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#selection-by-position" target="_blank">Indexing, selection, and filtering</a>

## Outliers

The majority of data values on the histograms above are concentrated on the far left of the figure. If you zoom in you will see there are a few isolated extreme or outlier yield values, which are masking the dominant pattern of the distribution. Detecting outliers is an important part of exploratory data analysis. 

Now that outliers have been detected we need to fix or remove them. A common way to detect outliers is to use a threshold based on percentile or standard deviation values. Here, we'll say an outlier is any value that is more or less than three standard deviations from the mean. 

In [ ]:
# Canola
df_canola = gdf.loc[gdf["Variety"] == "43Y23 RR", :]
print(f"There are {df_canola.shape[0]} canola rows BEFORE dropping outliers")
df_canola = df_canola.loc[(df_canola["DryYield"]-df_canola["DryYield"].mean()).abs() < (3*df_canola["DryYield"].std()), :]
print(f"There are {df_canola.shape[0]} canola rows AFTER dropping outliers")

# Wheat
df_wheat = gdf.loc[gdf["Variety"] == "Ninja", :]
print(f"There are {df_wheat.shape[0]} wheat rows BEFORE dropping outliers")
df_wheat = df_wheat.loc[(df_wheat["DryYield"]-df_wheat["DryYield"].mean()).abs() < (3*df_wheat["DryYield"].std()), :]
print(f"There are {df_wheat.shape[0]} wheat rows AFTER dropping outliers")

#### Recap quiz

<details>
    <summary><b>Is <code>df_canola = gdf.loc[gdf["Variety"] == "43Y23 RR", :]</code> an example of <em>selection by condition</em> or <em>selection by position</em>?</b></summary>
Selection by condition. Here, we're using a logical condition that evaluates to <code>True</code> for all rows where the value in the <code>Variety</code> column is <code>43Y23 RR</code>.
</details>

<p></p>

<details>
    <summary><b>What is the <code>abs()</code> function being used for in the conditional selection of rows whose <code>DryYield</code> value is more / less than three standard deviations from the mean?</b></summary>
<code>abs()</code> returns the absolute numeric value (so -2 would be returned as 2). This converts all negative deviations from the mean to positive and allows us to test for rows less than 3 standard deviations from the mean and select all rows within three standard deviations of the mean. 
</details>

<p></p>

<details>
    <summary><b>What is the <a href="https://pandas.pydata.org/docs/reference/api/pandas.concat.html" target="_blank">pandas <code>concat()</code></a> function used for?</b></summary>
    To combine (concatenate) <code>DataFrames</code> along an axis. Here, we specify the 0 axis which refers to rows to stack two `DataFrame`s on top of each other.  
</details>

In [ ]:
# combine filtered dfs
gdf_clean = pd.concat([df_canola, df_wheat], axis=0)

In [ ]:
fig = px.histogram(
    data_frame=gdf_clean, 
    x="DryYield", 
    facet_col="Variety", 
    marginal="box", 
    hover_data=["DryYield", "Elevation", "WetMass"])
fig.show()

Instead of dropping rows where there are extreme crop yield values, we can also replace outlier values with a more sensible value such as the mean.  

In [ ]:
mean_yield = gdf.loc[:, ["Variety", "DryYield"]].groupby(["Variety"]).mean()
mean_yield

In [ ]:
df_canola = gdf.loc[gdf["Variety"] == "43Y23 RR", :]
df_canola.loc[(df_canola["DryYield"]-df_canola["DryYield"].mean()).abs() > (3*df_canola["DryYield"].std()), "DryYield"] = mean_yield.iloc[0, 0]

df_wheat = gdf.loc[gdf["Variety"] == "Ninja", :]
df_wheat.loc[(df_wheat["DryYield"]-df_wheat["DryYield"].mean()).abs() > (3*df_wheat["DryYield"].std()), "DryYield"] = mean_yield.iloc[1, 0]

# combine filtered dfs
gdf_replaced = pd.concat([df_canola, df_wheat], axis=0)

In [ ]:
fig = px.histogram(
    data_frame=gdf_replaced, 
    x="DryYield", 
    facet_col="Variety", 
    marginal="box", 
    hover_data=["DryYield", "Elevation", "WetMass"])
fig.show()

Looking at the wheat yield histogram we can see there are a large number of zero or close to zero values. This is another strange artefact in the distribution of our data values. Are zero crop yield values actually no crop yield from the plant or a source of measurement error or other noise? If the latter, we should remove these noisy values. 

In [ ]:
df_canola = gdf_clean.loc[gdf_clean["Variety"] == "43Y23 RR", :]
print(f"The number of canola observations with yield values of zero or less is: {(df_canola['DryYield'] <= 0).sum()}")
df_wheat = gdf_clean.loc[gdf_clean["Variety"] == "Ninja", :]
print(f"The number of wheat observations with yield values of zero or less is: {(df_wheat['DryYield'] <= 0).sum()}")

Above, when setting the outlier values to their mean value, we demonstrated how can use `loc` on the left hand side of an expression to determine which values are set. 

Here, all rows in `df_canola` with a value more / less than three standard deviations from the mean are set to the mean value.

`df_canola.loc[(df_canola["DryYield"]-df_canola["DryYield"].mean()).abs() > (3*df_canola["DryYield"].std()), "DryYield"] = mean_yield.iloc[0, 0]`

#### Recap quiz

**We're going to drop zero `DryYield` values as these values are noisy. Can you use a *selection by condition* operation to subset the `GeoDataFrame` `gdf_clean` to select only rows where `DryYield` is greater than zero?**


In [ ]:
## Add code here ##

<details>
    <summary><b>answer</b></summary>

```{python}
print("Shape of DataFrame before dropping zero yield values:")
print(gdf_clean.shape)
gdf_dropped_zero = gdf_clean.loc[gdf_clean["DryYield"] > 0, :]
print("Shape of DataFrame after dropping zero yield values:")
print(gdf_dropped_zero.shape)
```
</details>

<p></p>

**Can you generate a histogram visualising the distribution of `DryYield` values after dropping zero values? Use the `Variety` column to generate faceted subplots for each wheat and canola.**

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer</b></summary>

```{python}
fig = px.histogram(
    data_frame=gdf_dropped_zero, 
    x="DryYield", 
    facet_col="Variety", 
    marginal="box", 
    hover_data=["DryYield", "Elevation", "WetMass"])
fig.show()
```
</details>

After removing zero values the distribution of our crop yield values should look more sensible and relatively normally distributed.

### 2D histograms

We can use 2D histograms or density heatmaps to look at the distribution of two variables together. 2D hisograms are a useful complement to scatter plots when you have a large number of observations. Here, colour is used to represent the distribution of data values as opposed to the height of rectangular bars on a histogram.

Let's create 2D histograms to visualise the relationship between vegetation indices and canola crop yield. Note, you'll need to have completed the recap quiz above to generate `gdf_dropped_zero`. 

In [ ]:
fig = px.density_heatmap(
    data_frame=gdf_dropped_zero.loc[gdf_dropped_zero["Variety"] == "43Y23 RR", :], 
    x="DryYield", 
    y="gndvi", 
    marginal_x="box", 
    marginal_y="box",
    range_y=[0.4, 0.8])
fig.show()

In [ ]:
fig = px.density_heatmap(
    data_frame=gdf_dropped_zero.loc[gdf_dropped_zero["Variety"] == "43Y23 RR", :], 
    x="DryYield", 
    y="ndyi", 
    marginal_x="box", 
    marginal_y="box",
    range_y=[0.1, 0.5])
fig.show()

### Violin plots

One of the limits of using histograms to visualise distributions is the size of the bins affects the distribution of data values. An alternative approach to visualising a distribution is to use a violin plot. 

Violin plots use density and box plots to visualise distributions, which look similar to violins. The density is the probability of an observation taking on a certain value and is plotted as a smooth curve. Areas where the curve is fatter indicate a higher probability that an observation will take that value. Box plots display the 25th, 50th, and 75th percentile values.

In [ ]:
fig = px.violin(
    gdf_dropped_zero, 
    y="DryYield", 
    x="Variety", 
    color="Variety", 
    box=True, 
    points="outliers", 
    hover_data=["DryYield", "gndvi"])
fig.show()

#### Recap quiz

**Can you create violin plots to visualise the GNDVI and NDYI values for each crop type?**

In [ ]:
## ADD GNDVI VIOLIN PLOT CODE HERE ##

<details>
    <summary><b>answer</b></summary>
    
```{python}
fig = px.violin(
    gdf_dropped_zero, 
    y="gndvi", 
    x="Variety", 
    color="Variety", 
    box=True, 
    points="outliers", 
    hover_data=["DryYield", "gndvi"])
fig.show()
```
</details>

In [ ]:
## ADD NDYI VIOLIN PLOT CODE HERE ##

<details>
    <summary><b>answer</b></summary>
    
```{python}
fig = px.violin(
    gdf_dropped_zero, 
    y="ndyi", 
    x="Variety", 
    color="Variety", 
    box=True, 
    points="outliers", 
    hover_data=["DryYield", "ndyi"])
fig.show()
```
</details>